### **기본 세팅**
---

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install tqdm
!pip install catboost

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import os
import sys
import math
import random

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from matplotlib import font_manager, rc
rc('font', family='NanumBarunGothic') 

seed = 0

data_path = "/content/drive/Shareddrives/Parking_pred/data/Encoded/"
result_path = "/content/drive/Shareddrives/Parking_pred/jihoon/result/"
result_file = "mh_no_area.csv"

train = pd.read_excel(data_path + "train.xlsx")
test = pd.read_excel(data_path + "test.xlsx")
api_lh = pd.read_excel(data_path + "openapi_lh.xlsx")
api_mh = pd.read_excel(data_path + "openapi_mh.xlsx")
age_gender = pd.read_excel(data_path + "age_gender_info.xlsx")

submission = pd.read_excel(data_path + "sample_submission.xlsx")

train.shape, test.shape, api_lh.shape, api_mh.shape

((2952, 15), (1022, 14), (5604, 8), (13708, 10))

### **오류 데이터 처리**
---

In [ ]:
train_code = ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
test_code = ['C2335', 'C1327']
submission_code = ['C2335', 'C1327', 'C2675']
for c in train_code:
    train = train[train['단지코드'] != c]

# for c in test_code:
#     test = test[test['단지코드'] != c]

# for c in submission_code:
#     submission = submission[submission['code'] != c]

train.shape, test.shape, submission.shape

((2896, 15), (1022, 14), (150, 2))

In [ ]:
c2085 = train[train['단지코드'] == 'C2085'].index
train.loc[c2085, '단지코드'] = 'C1397'

c1397 = train[train['단지코드'] == 'C1397'].index
train.loc[c1397, '총세대수'] = 1339

In [ ]:
c2461 = train[train['단지코드'] == 'C2431'].index
train.loc[c2461, '단지코드'] = 'C1649'

c1649 = train[train['단지코드'] == 'C1649'].index
train.loc[c1649, '총세대수'] = 1047
train.loc[c1649, '등록차량수'] = 1214

In [ ]:
c2675 = test[test['단지코드'] == 'C2675']
c2675['등록차량수'] = 1279
c2675['단지코드'] = 'C1036'
c2675

# test = test[test['단지코드'] != 'C2675']

train = train.append(c2675)
train = train.reset_index(drop=True)
c1036 = train[train['단지코드'] == 'C1036'].index
train.loc[c1036, '총세대수'] = 1254

test[test['단지코드'] == 'C2675'], train[train['단지코드'] == 'C1036']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(      단지코드  총세대수  ... 도보 10분거리 내 버스정류장 수 단지내주차면수
 579  C2675   512  ...                  3    1016
 580  C2675   512  ...                  3    1016
 581  C2675   512  ...                  3    1016
 582  C2675   512  ...                  3    1016
 583  C2675   512  ...                  3    1016
 584  C2675   512  ...                  3    1016
 585  C2675   512  ...                  3    1016
 586  C2675   512  ...                  3    1016
 
 [8 rows x 14 columns],
        단지코드  총세대수 임대건물구분  ... 도보 10분거리 내 버스정류장 수 단지내주차면수  등록차량수
 1490  C1036  1254    아파트  ...                3.0    1016   1279
 1491  C1036  1254    아파트  ...                3.0    1016   1279
 1492  C1036  1254    아파트  ...                3.0    1016   1279
 1493  C1036  1254    아파트  ...                3.0    1016   1279
 1494  C1036  1254    아파트  ...                3.0    1016   1279
 1495  C1036  1254    아파트  ...                3.0    1016   1279
 1496  C1036  1254    아파트  ...                3.0    1016   1279
 289

In [ ]:
def get_mod_miss_households(data):
    total = data.iloc[0]['총세대수']
    sum_hh = data['전용면적별세대수'].sum()
    miss = total - sum_hh
    prop = (data['전용면적별세대수'] / sum_hh).tolist()
    miss_prop = [int(p * miss) for p in prop]
    int_miss = miss - sum(miss_prop)
    for i in range(int_miss):
        miss_prop[(i)%len(miss_prop)] += 1

    return miss_prop

total_miss = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804',
                'C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
                'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
                'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
                'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
                'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']

for tm in total_miss:
    if train[train['단지코드'] == tm].shape[0] == 0:
        if test[test['단지코드'] == tm].shape[0] == 0:
            print("Empty: {}".format(tm))
            continue
        else:
            mod = get_mod_miss_households(test[test['단지코드'] == tm])
            idx = test[test['단지코드'] == tm].index
            test.loc[idx, '전용면적별세대수'] += mod
            # print(test[test['단지코드'] == tm][["총세대수", "전용면적별세대수"]])
    else:
        mod = get_mod_miss_households(train[train['단지코드'] == tm])
        idx = train[train['단지코드'] == tm].index
        train.loc[idx, '전용면적별세대수'] += mod
        # print(train[train['단지코드'] == tm][["총세대수", "전용면적별세대수"]])

train.shape, test.shape

Empty: C1218
Empty: C1894


((2904, 15), (1022, 14))

In [ ]:
train.reset_index(drop = True, inplace=True)
test.reset_index(drop = True, inplace=True)
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,C1036,1254,아파트,경기도,국민임대,46.90,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2900,C1036,1254,아파트,경기도,국민임대,46.90,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2901,C1036,1254,아파트,경기도,국민임대,47.40,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2902,C1036,1254,아파트,경기도,국민임대,51.93,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


In [ ]:
## 지역
api_mh = api_mh[api_mh['지역'] != "인천광역시"]
api_mh

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,50년임대,38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,50년임대,40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,50년임대,50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,50년임대,33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,50년임대,32.7600,534,13510000,178000,263
...,...,...,...,...,...,...,...,...,...,...
13703,30699399,71,아파트,제주특별자치도,10년임대,84.7200,71,0,0,306
13704,30699399,71,아파트,제주특별자치도,10년임대,84.8200,71,0,0,306
13705,30699399,71,아파트,제주특별자치도,10년임대,84.8400,71,0,0,306
13706,30699399,71,아파트,제주특별자치도,10년임대,84.8500,71,0,0,306


In [ ]:
## 공급유형
print(train['공급유형'].unique())
print(api_mh['공급유형'].unique())

idx_5rent = api_mh[api_mh['공급유형'] == "5년임대"].index
api_mh.loc[idx_5rent, '공급유형'] = "공공임대(5년)"
idx_10rent = api_mh[api_mh['공급유형'] == "10년임대"].index
api_mh.loc[idx_10rent, '공급유형'] = "공공임대(10년)"
idx_50rent = api_mh[api_mh['공급유형'] == "50년임대"].index
api_mh.loc[idx_50rent, '공급유형'] = "공공임대(50년)"

api_mh = api_mh[api_mh['공급유형'] != '매입임대']

api_mh

['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']
['50년임대' '장기전세' '국민임대' '매입임대' '행복주택' '영구임대' '10년임대' '5년임대']


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,공공임대(50년),38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,공공임대(50년),40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,공공임대(50년),50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,공공임대(50년),33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,공공임대(50년),32.7600,534,13510000,178000,263
...,...,...,...,...,...,...,...,...,...,...
13703,30699399,71,아파트,제주특별자치도,공공임대(10년),84.7200,71,0,0,306
13704,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8200,71,0,0,306
13705,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8400,71,0,0,306
13706,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8500,71,0,0,306


In [ ]:
# 단지내주차면수 0
api_mh = api_mh[api_mh['단지내주차면수'] != 0]
api_mh

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,공공임대(50년),38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,공공임대(50년),40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,공공임대(50년),50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,공공임대(50년),33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,공공임대(50년),32.7600,534,13510000,178000,263
...,...,...,...,...,...,...,...,...,...,...
13703,30699399,71,아파트,제주특별자치도,공공임대(10년),84.7200,71,0,0,306
13704,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8200,71,0,0,306
13705,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8400,71,0,0,306
13706,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8500,71,0,0,306


In [ ]:
import statsmodels.api as sm

def display_total_parknums_summary(data, seq, fig):
    total = []
    park_nums = []
    for i, code in enumerate(data['단지코드'].unique()):
        temp = data[data['단지코드']==code]
        temp.index = range(temp.shape[0])
        total.append(temp.loc[0, "총세대수"])
        park_nums.append(temp.loc[0, "단지내주차면수"])
    regline = sm.OLS(park_nums, sm.add_constant(total)).fit().fittedvalues
    fig.add_trace( go.Scatter(x=total, y=park_nums, mode="markers", 
                              marker_color=px.colors.qualitative.Plotly[seq]), row=seq+1, col=1)
    fig.add_trace(go.Scatter(x=total, y=regline,
                            mode="lines", marker_color="black"), row=seq+1, col=1)
    fig.add_trace(go.Box(y= np.array(park_nums) / np.array(total),
                            marker_color=px.colors.qualitative.Plotly[seq]), row=seq+1, col=2)
    
fig = make_subplots(3,2, column_widths=[0.7, 0.3])
fig.update_layout(width=1000, height=1000)
display_total_parknums_summary(train, 0, fig)
display_total_parknums_summary(test, 1, fig)
display_total_parknums_summary(api_mh, 2, fig)
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
train = train[train['단지내주차면수']/train['총세대수'] < 1.289]
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,C1036,1254,아파트,경기도,국민임대,46.90,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2900,C1036,1254,아파트,경기도,국민임대,46.90,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2901,C1036,1254,아파트,경기도,국민임대,47.40,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2902,C1036,1254,아파트,경기도,국민임대,51.93,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


In [ ]:
api_mh = api_mh[api_mh['단지내주차면수']/api_mh['총세대수'] < 1.289]
api_mh

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
4,30583336,534,아파트,서울특별시,공공임대(50년),32.76,534,13510000,178000,263
5,30583314,363,아파트,서울특별시,공공임대(50년),32.99,363,10140000,144000,163
10,30583298,550,아파트,서울특별시,공공임대(50년),32.93,550,10600000,149200,205
14,31144292,64,아파트,서울특별시,공공임대(50년),38.20,64,34390000,178700,48
15,31144292,64,아파트,서울특별시,공공임대(50년),42.42,64,38050000,197700,48
...,...,...,...,...,...,...,...,...,...,...
13696,30699493,511,아파트,제주특별자치도,국민임대,51.61,511,18968000,207380,413
13697,30699493,511,아파트,제주특별자치도,국민임대,51.86,511,18968000,209910,413
13698,30699493,511,아파트,제주특별자치도,국민임대,51.87,511,18968000,209910,413
13699,30699493,511,아파트,제주특별자치도,국민임대,51.99,511,18968000,209910,413


### **단지 통합**

In [ ]:
columns = ['단지코드', '총세대수', '지역', '공급유형', '단지내주차면수', '등록차량수']
cat_features = ['지역', '공급유형']
tmp_train = train[columns].copy()
tmp_test = test[columns[:-1]].copy()
tmp_mh = api_mh[columns[:-1]].copy()

In [ ]:
for cf in cat_features:
    map = {}
    for i, ele in enumerate(tmp_train[cf].unique()):
        map[ele] = i
    tmp_train[cf] = tmp_train[cf].map(map)
    tmp_test[cf] = tmp_test[cf].map(map)
    tmp_mh[cf] = tmp_mh[cf].map(map)

In [ ]:
tmp_train

,단지코드,총세대수,지역,공급유형,단지내주차면수,등록차량수
0,C2515,545,0,0,624,205
1,C2515,545,0,0,624,205
2,C2515,545,0,0,624,205
3,C2515,545,0,0,624,205
4,C2515,545,0,0,624,205
...,...,...,...,...,...,...
2899,C1036,1254,2,0,1016,1279
2900,C1036,1254,2,0,1016,1279
2901,C1036,1254,2,0,1016,1279
2902,C1036,1254,2,0,1016,1279


In [ ]:
tmp_mh

,단지코드,총세대수,지역,공급유형,단지내주차면수
4,30583336,534,14,1,263
5,30583314,363,14,1,163
10,30583298,550,14,1,205
14,31144292,64,14,1,48
15,31144292,64,14,1,48
...,...,...,...,...,...
13696,30699493,511,8,0,413
13697,30699493,511,8,0,413
13698,30699493,511,8,0,413
13699,30699493,511,8,0,413


In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()
new_mh = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(tmp_train['단지코드'].unique())):
    temp = tmp_train[tmp_train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    new_train.loc[i, '공급유형'] = temp['공급유형'].value_counts().index.tolist()[0]
    
for i, code in tqdm(enumerate(tmp_test['단지코드'].unique())):
    temp = tmp_test[tmp_test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns[:-1]:
        new_test.loc[i, col] = temp.loc[0, col]

    new_test.loc[i, '공급유형'] = temp['공급유형'].value_counts().index.tolist()[0]

for i, code in tqdm(enumerate(tmp_mh['단지코드'].unique())):
    temp = tmp_mh[tmp_mh['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns[:-1]:
        new_mh.loc[i, col] = temp.loc[0, col]

    new_mh.loc[i, '공급유형'] = temp['공급유형'].value_counts().index.tolist()[0]

409it [00:01, 212.54it/s]
150it [00:00, 263.28it/s]
1149it [00:03, 367.26it/s]


In [ ]:
new_train

,단지코드,총세대수,지역,공급유형,단지내주차면수,등록차량수
0,C2515,545.0,0.0,0.0,624.0,205.0
1,C1407,1216.0,1.0,0.0,1285.0,1064.0
2,C1945,755.0,2.0,0.0,734.0,730.0
3,C1470,696.0,3.0,0.0,645.0,553.0
4,C1898,566.0,3.0,0.0,517.0,415.0
...,...,...,...,...,...,...
404,C2586,90.0,8.0,8.0,66.0,57.0
405,C2035,492.0,4.0,0.0,521.0,246.0
406,C2020,40.0,7.0,8.0,25.0,19.0
407,C2437,90.0,10.0,2.0,30.0,16.0


In [ ]:
new_test

,단지코드,총세대수,지역,공급유형,단지내주차면수
0,C1072,754.0,2.0,0.0,683.0
1,C1128,1354.0,2.0,0.0,1216.0
2,C1456,619.0,7.0,0.0,547.0
3,C1840,593.0,3.0,0.0,543.0
4,C1332,1297.0,2.0,0.0,1112.0
...,...,...,...,...,...
145,C2456,349.0,8.0,0.0,270.0
146,C1266,596.0,10.0,0.0,593.0
147,C2152,120.0,4.0,2.0,40.0
148,C1267,675.0,0.0,0.0,467.0


In [ ]:
new_mh

,단지코드,총세대수,지역,공급유형,단지내주차면수
0,30583336.0,534.0,14.0,1.0,263.0
1,30583314.0,363.0,14.0,1.0,163.0
2,30583298.0,550.0,14.0,1.0,205.0
3,31144292.0,64.0,14.0,1.0,48.0
4,31015003.0,432.0,14.0,1.0,346.0
...,...,...,...,...,...
1144,30700225.0,620.0,8.0,0.0,498.0
1145,30699587.0,212.0,8.0,0.0,163.0
1146,30699517.0,382.0,8.0,0.0,306.0
1147,30699504.0,321.0,8.0,0.0,259.0


### **학습 및 추론**
--- 


In [ ]:
X_train = new_train.iloc[:, 1:-1]
Y_train = new_train.iloc[:,-1]
mh_train = new_mh.iloc[:,1:]
x_test = new_test.iloc[:,1:]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, 
                                                      random_state=seed, shuffle=True)

In [ ]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(
        loss_function='MAE',
        n_estimators=300, 
        learning_rate=0.05, 
        random_state=seed     
    )
cat.fit(x_train, y_train, eval_set=(x_valid, y_valid), use_best_model=True)

0:	learn: 281.8287450	test: 263.3268281	best: 263.3268281 (0)	total: 47.3ms	remaining: 14.1s
1:	learn: 272.3709008	test: 255.1213705	best: 255.1213705 (1)	total: 48.8ms	remaining: 7.26s
2:	learn: 263.1093872	test: 248.8806174	best: 248.8806174 (2)	total: 49.9ms	remaining: 4.93s
3:	learn: 255.0169530	test: 242.2785367	best: 242.2785367 (3)	total: 50.9ms	remaining: 3.77s
4:	learn: 246.5781243	test: 234.3366626	best: 234.3366626 (4)	total: 51.9ms	remaining: 3.06s
5:	learn: 238.4167262	test: 227.6271010	best: 227.6271010 (5)	total: 52.9ms	remaining: 2.59s
6:	learn: 231.3846941	test: 221.7667026	best: 221.7667026 (6)	total: 54ms	remaining: 2.26s
7:	learn: 225.1022072	test: 217.6276660	best: 217.6276660 (7)	total: 55ms	remaining: 2.01s
8:	learn: 219.2337110	test: 213.3552740	best: 213.3552740 (8)	total: 56.1ms	remaining: 1.81s
9:	learn: 213.2190215	test: 207.1883167	best: 207.1883167 (9)	total: 57.1ms	remaining: 1.65s
10:	learn: 207.1013924	test: 202.6281744	best: 202.6281744 (10)	total: 58.

In [ ]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_valid, cat.predict(x_valid))
score

116.4029833340018

In [ ]:
pred = cat.predict(mh_train)

In [ ]:
mh_train['등록차량수'] = pred
mh_train

,총세대수,지역,공급유형,단지내주차면수,등록차량수
0,534.0,14.0,1.0,263.0,315.935619
1,363.0,14.0,1.0,163.0,215.006007
2,550.0,14.0,1.0,205.0,306.208278
3,64.0,14.0,1.0,48.0,125.574465
4,432.0,14.0,1.0,346.0,341.402275
...,...,...,...,...,...
1144,620.0,8.0,0.0,498.0,500.589951
1145,212.0,8.0,0.0,163.0,152.322534
1146,382.0,8.0,0.0,306.0,299.348032
1147,321.0,8.0,0.0,259.0,273.060105


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = new_train['총세대수'], y=new_train['단지내주차면수'], 
                         mode="markers", marker_color=px.colors.qualitative.Plotly[0], 
                         name="train"))
fig.add_trace(go.Scatter(x = mh_train['총세대수'], y=mh_train['단지내주차면수'], 
                         mode="markers", marker_color=px.colors.qualitative.Plotly[1], 
                         name="mh"))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = new_train['총세대수'], y=new_train['등록차량수'], 
                         mode="markers", marker_color=px.colors.qualitative.Plotly[0], 
                         name="train"))
fig.add_trace(go.Scatter(x = mh_train['총세대수'], y=mh_train['등록차량수'], 
                         mode="markers", marker_color=px.colors.qualitative.Plotly[1], 
                         name="mh"))
fig.show()

In [ ]:
x_train = x_train.append(mh_train.iloc[:, :-1])
y_train = y_train.append(mh_train.iloc[:, -1])
x_train

,총세대수,지역,공급유형,단지내주차면수
341,938.0,12.0,0.0,1117.0
367,450.0,11.0,8.0,318.0
380,442.0,13.0,4.0,533.0
75,775.0,11.0,0.0,738.0
64,1144.0,6.0,0.0,950.0
...,...,...,...,...
1144,620.0,8.0,0.0,498.0
1145,212.0,8.0,0.0,163.0
1146,382.0,8.0,0.0,306.0
1147,321.0,8.0,0.0,259.0


In [ ]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(
        loss_function='MAE',
        n_estimators=800, 
        learning_rate=0.05, 
        random_state=seed     
    )
cat.fit(x_train, y_train, eval_set=(x_valid, y_valid), use_best_model=True)

0:	learn: 243.8046322	test: 280.1807478	best: 280.1807478 (0)	total: 1.42ms	remaining: 1.14s
1:	learn: 234.1032920	test: 270.6756071	best: 270.6756071 (1)	total: 3.28ms	remaining: 1.31s
2:	learn: 224.4390801	test: 262.0940794	best: 262.0940794 (2)	total: 4.71ms	remaining: 1.25s
3:	learn: 215.4863180	test: 253.9250529	best: 253.9250529 (3)	total: 5.96ms	remaining: 1.19s
4:	learn: 206.4459150	test: 246.4232998	best: 246.4232998 (4)	total: 7.39ms	remaining: 1.18s
5:	learn: 198.1766899	test: 238.7101170	best: 238.7101170 (5)	total: 8.78ms	remaining: 1.16s
6:	learn: 190.7101051	test: 232.7797363	best: 232.7797363 (6)	total: 10.2ms	remaining: 1.15s
7:	learn: 183.4981199	test: 226.4284003	best: 226.4284003 (7)	total: 11.7ms	remaining: 1.16s
8:	learn: 177.5035842	test: 222.4461165	best: 222.4461165 (8)	total: 13.1ms	remaining: 1.15s
9:	learn: 171.1140751	test: 216.9887655	best: 216.9887655 (9)	total: 14.4ms	remaining: 1.14s
10:	learn: 164.9481280	test: 212.7215171	best: 212.7215171 (10)	total:

In [ ]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_valid, cat.predict(x_valid))
score

114.82303784356661

In [ ]:
pred = cat.predict(x_test)

In [ ]:
submission['num'] = pred

In [ ]:
# submission.to_csv(result_path + result_file, index=False)